In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib

from nilearn.plotting import plot_stat_map, plot_anat, plot_img
from nilearn.image import mean_img, image, new_img_like
from nilearn.input_data import NiftiMasker

from nistats.first_level_model import FirstLevelModel
from nistats.reporting import plot_design_matrix

from fc_config import *

from glm_timing import glm_timing

from preprocess_library import meta

In [2]:
sub = 1
subj = meta(sub)
phase = 'fear_conditioning'
events = glm_timing(sub,phase).phase_events()
func = os.path.join(subj.bold_dir,nifti_paths[phase])

In [3]:
masker = NiftiMasker(mask_img=subj.brainmask, smoothing_fwhm=5, detrend=True,
                    high_pass=0.0078, t_r=2, standardize=True).fit(imgs=func)

In [4]:
glm = FirstLevelModel(t_r=2, mask=masker, slice_time_ref=.5, hrf_model='glover + derivative', signal_scaling=False, verbose=2, minimize_memory=False)

In [7]:
motion_confounds = np.loadtxt(os.path.join(subj.bold_dir, phase2rundir[phase], 'motion_assess', 'confound.txt'))
if len(motion_confounds) > 0:

    if len(motion_confounds.shape) == 2:
        _columns = range(motion_confounds.shape[1])
    elif len(motion_confounds.shape) == 1:
        _columns = range(1)

    motion_confounds_df = pd.DataFrame(motion_confounds, columns=_columns)

else: 
    motion_confounds_df = None

/Users/ach3377/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: loadtxt: Empty input file: "/Users/ach3377/Db_lpl/STUDY/FearCon/Sub001/bold/day1/run002/motion_assess/confound.txt"
  """Entry point for launching an IPython kernel.


In [8]:
glm = glm.fit(func, events, confounds=motion_confounds_df)
design_matrix = glm.design_matrices_[0]

In [ ]:
contrast_matrix = np.eye(design_matrix.shape[1])

#fill out the rest of the contrast matrix with variables from the design matrix
contrasts = dict([(column, contrast_matrix[i])
    for i, column in enumerate(design_matrix.columns)])

CSplus_minus_CSmin = contrasts['CS+'] - contrasts['CS-']
contrast_ = CSplus_minus_CSmin

In [ ]:
glm.compute_contrast(contrast_, stat_type='t', output_type='z_score')

In [ ]:
glm.results_[0][0].predicted

In [15]:
type(glm.results_)

list